# Répartition des élèves dans les MIG
(Notebook écrit à 99,9% par Matthieu Colin, merci à lui !!)

## Comment ça marche ?

L'algorithme utilisé est [l'algorithme hongrois](https://fr.wikipedia.org/wiki/Algorithme_hongrois), ie avec scipy la méthode `linear_sum_assignment`.<br>
Pour fonctionner, il a besoin d'autant de personnes que de tâches (ici, des places dans les MIG). <br>
Ce que ce notebook fait, c'est attribuer chaque place individuelle à un étudiant (même si en pratique, la place 1 du MIG Océan et la place 10 du MIG Océan sont exactement les mêmes, c'est juste pour que l'algo tourne).

## Comment l'utiliser ?

Il faut un fichier `voeux.csv` avec une colonne Nom (sous format Prénom Nom pour que tout fonctionne, mais ça marche même sans) et ensuite, les voeux classés de $1$ à $n$ où $n$ est le nombre de MIG. Google Forms le fait tout seul normalement !

Ensuite, changer les noms des MIG dans la variable `NAMES`.

Ensuite, appuyer sur `Run all` et un fichier `repartition.csv` va pop avec la répartition !

In [1]:
from scipy.optimize import linear_sum_assignment
import numpy as np
import pandas as pd
import random

In [2]:
# CONSTANTES A MODIFIER

# Nb d'étudiants
N = 150
# Dictionnaire des correspondances
CORRESP = {1: "R-Source", 2: "Mode", 3: "E-Mix", 4:"ALEF", 5: "Océan", 6: "Solaire", 7: "Santé", 8: "Verre", 9:"Aéro"}
# Dictionnaire des nombres de places par MIG
NB_PLACES = {1: 17, 2: 16, 3: 14, 4: 16, 5: 18, 6: 17, 7: 17, 8: 17, 9: 18}
# Numéro de la première colonne qui correspond à un voeu (ATTENTION COLONNE 1 du Google Sheet = colonne 0 pour le notebook)
NUM_PREMIERE_COLONNE = 3
# Noms des colonnes du Google Sheet
NAMES = ["Date", "Nom", "Prenom", "R-Source", "Mode", "E-Mix", "ALEF", "Océan", "Solaire", "Santé", "Verre", 'Aéro']

In [3]:
# CREATION DE CONSTANTES A NE PAS MODIFIER

# Le code est vraiment pas beau mais menfou ça marche
CORRESPONDANCES = np.empty(N, dtype=object)
compteur_places = 0
for x in NB_PLACES.keys():
    for j in range(NB_PLACES[x]):
        CORRESPONDANCES[compteur_places] = CORRESP[x]
        compteur_places += 1

In [4]:
# CHANGER LES NOMS DES MIG

df = pd.read_csv("voeux.csv", names=NAMES, sep = ",", encoding='latin-1')

df

,Date,Nom,Prenom,R-Source,Mode,E-Mix,ALEF,Océan,Solaire,Santé,Verre,Aéro
0,gg,Di Giorgi,Louis,9,3,4,7,5,8,1,6,2
1,gg,Losantos,SolÃ¨ne,6,9,8,5,4,2,1,7,3
2,gg,BERAUD,GrÃ©goire,1,9,5,3,8,2,7,6,4
3,gg,PÃ©renne,RÃ©mi,5,9,6,7,1,8,2,3,4
4,gg,Deltour,Arnaud,8,9,5,4,2,3,1,7,6
...,...,...,...,...,...,...,...,...,...,...,...,...
145,gg,Babin,TimothÃ©e,2,7,9,4,6,5,3,1,8
146,gg,Porre,EugÃ©nie,3,4,8,6,7,9,5,2,1
147,gg,Michaud,Baptiste,6,8,9,5,3,4,1,7,2
148,gg,RITTANO,Osman,6,2,8,5,3,4,1,9,7


In [5]:
def random_ordered_matx() :
    
    #MELANGE DE L'ORDRE DES ELEVES

    df_shuffle = df.sample(frac=1).reset_index(drop=True)
    
    #REMPLISSAGE DE LA MATRICE
    
    mat = np.zeros((N, N), dtype=np.uint8)
        
    for i in range(N):
        compteur_places = 0
        for k in range(len(NB_PLACES)):
            for j in range(compteur_places, compteur_places+NB_PLACES[k+1]): # k+1 car le compte des voeux commence à 1 et non à 0
                mat[i][j] = df_shuffle.iloc[i, 3+k] # 3+k car les voeux commencent à partir de la 4ème colonne du Google Forms, le modifier selon 
                compteur_places += 1
    return mat, df_shuffle

In [6]:
# Calcul des gens qui ont eu un voeu supérieur à un seuil donné

def stats(df, row_ind, col_ind, visu=False) :

    VICTIMES = 5
    PADBOL = 4
    TANTPIS = 3
    PRESQUE = 2
    THRESHOLDS = [5,4,3,2]

    counts = {5:0, 4:0, 3:0, 2:0}
    names = {5:[], 4:[], 3:[], 2:[]}

    for i in range(N):
        for thres in THRESHOLDS :
            if df.loc[i, CORRESPONDANCES[col_ind[i]]] == thres or (thres == 5 and df.loc[i, CORRESPONDANCES[col_ind[i]]] >= 5) :
                counts[thres] += 1
                names[thres].append(df.loc[i, "Nom"]+ " " +df.loc[i, "Prenom"])

    if visu :

        print(f"""
            Victimes ({counts[VICTIMES]}) : {names[VICTIMES]}
            Pas de bol ({counts[PADBOL]}) : {names[PADBOL]}
            Tant pis ({counts[TANTPIS]}) : {names[TANTPIS]}
            Presque ({counts[PRESQUE]}) : {names[PRESQUE]}
            Et {N-counts[VICTIMES]-counts[PADBOL]-counts[TANTPIS]-counts[VICTIMES]} heureux !
        """)

    return ( counts[VICTIMES], counts[PADBOL] , counts[TANTPIS] , counts[VICTIMES] )

In [7]:
def min_index(l):
    val_min = l[0]
    ind_min = 0
    for i in range(1,len(l)):
        v = l[i]
        if v < val_min :
            val_min = v
            ind_min = i
    return ind_min

In [8]:
def optimize(nb_essais) :
    stat_list = []
    for essai in range(nb_essais) :

        #TRAITEMENT (~190ms)

        mat, df_shuffle = random_ordered_matx() #Mélange de l'ordre des élèves (~140ms)
        row_ind, col_ind = linear_sum_assignment(mat) #Application de l'algorithme (~50ms)

        #POST-TRAITEMENT (~50ms)

        df_shuffle['Repartition'] = [CORRESPONDANCES[i] for i in col_ind]
        df_shuffle["Rang de l'affectation"] = np.empty(N, dtype=object)
        for i in range(N):
            affect = df_shuffle.loc[i, "Repartition"]
            df_shuffle.loc[i, "Rang de l'affectation"] = df_shuffle.loc[i, affect]

        order = df_shuffle["Nom"].str.split(" ").str[1:].apply(" ".join).str.upper().sort_values()
        order.index
        
        #EXPORT (~50ms)
        
        (df_shuffle[["Nom", "Prenom","Repartition", "Rang de l'affectation"]].sort_values(by=["Nom", "Prenom"], ignore_index=True)).iloc[order.index].to_csv(f"Repartitions/repartition2024_jet{essai}.csv")

        stat_list.append(stats(df_shuffle, row_ind, col_ind))
    
    paps_best_index = min_index(stat_list)
    best_res = stat_list[paps_best_index]
    index_optimaux = [i for i in range(len(stat_list)) if stat_list[i]==best_res]
    winner = index_optimaux[random.randrange(len(index_optimaux))] # Tirage au sort parmi ceux qui ont le meilleur combo stat
    print(f"Le meilleur résultat retenu est la répartition n°{winner} ({len(index_optimaux) - 1} doublons)")
    a,b,c,d = best_res
    print(f"""
        Victimes : {a}
        Pas de bol : {b}
        Tant pis {c}
        Presque : {d}
        Et {N-a-b-c-d} heureux !
    """)
    print(index_optimaux)
    return pd.read_csv(f"Repartitions/repartition2024_jet{winner}.csv", sep = ",", encoding='latin-1')[["Nom", "Prenom", "Repartition", "Rang de l'affectation"]]

In [9]:
optimize(1000)

Le meilleur résultat retenu est la répartition n°841 (3 doublons)

        Victimes : 1
        Pas de bol : 3
        Tant pis 12
        Presque : 1
        Et 133 heureux !
    
[176, 415, 442, 841]


,Nom,Prenom,Repartition,Rang de l'affectation
0,Ahrend,Laurentin-wilhelm,Verre,1
1,Mazelier,CÃÂ´me,AÃ©ro,1
2,Marzin,Ewen,Solaire,1
3,Martinez,Adrien,R-Source,1
4,Martin FrÃÂ©our,Esteban,ALEF,2
...,...,...,...,...
145,Bardinet,Guillaume,Solaire,1
146,Weisman,Nathan,AÃ©ro,1
147,Vandersippe,Gabriel,OcÃ©an,1
148,Lauret,Arthur,AÃ©ro,2
